# <center> Topic Modeling with Gensim </center>
---

In [1]:
from sklearn.datasets import fetch_20newsgroups

categories = ['alt.atheism', 'talk.religion.misc', 'comp.graphics', 'sci.space']

newsgroups_train = fetch_20newsgroups(subset='train',
                                      remove=('headers', 'footers', 'quotes'),
                                      categories=categories)

In [2]:
from nltk.corpus import stopwords
cachedStopWords = stopwords.words("english")

from nltk import word_tokenize
from nltk.stem.porter import PorterStemmer
import re

In [3]:
def tokenize(text):
    min_length = 3   # 최소 단어 크기
                     # nltk의 tokenizer ->  word 추출 -> 소문자로 변환
    words = map(lambda word: word.lower(), word_tokenize(text))
    words = [word for word in words if word not in cachedStopWords]
 
    tokens = (list(map(lambda token: PorterStemmer().stem(token),words))) # 알파벳 추출

    p = re.compile('[a-zA-Z]+');
    filtered_tokens = list(filter (lambda token: p.match(token) and len(token) >= min_length,tokens))
    return filtered_tokens

In [4]:
docs = [tokenize(doc) for doc in newsgroups_train.data]

In [5]:
print(newsgroups_train.data[0])
print(docs[0])

Hi,

I've noticed that if you only save a model (with all your mapping planes
positioned carefully) to a .3DS file that when you reload it after restarting
3DS, they are given a default position and orientation.  But if you save
to a .PRJ file their positions/orientation are preserved.  Does anyone
know why this information is not stored in the .3DS file?  Nothing is
explicitly said in the manual about saving texture rules in the .PRJ file. 
I'd like to be able to read the texture rule information, does anyone have 
the format for the .PRJ file?

Is the .CEL file format available from somewhere?

Rych
['notic', 'save', 'model', 'map', 'plane', 'posit', 'care', 'file', 'reload', 'restart', 'given', 'default', 'posit', 'orient', 'save', 'file', 'positions/orient', 'preserv', 'anyon', 'know', 'inform', 'store', 'file', 'noth', 'explicitli', 'said', 'manual', 'save', 'textur', 'rule', 'file', 'like', 'abl', 'read', 'textur', 'rule', 'inform', 'anyon', 'format', 'file', 'file', 'format', 'a

Gensim으로 topic modeling을 하기 위해서는 Dictionary, Corpus, Model의 3단계를 거쳐야 함

In [6]:
from gensim.corpora import Dictionary

dictionary = Dictionary(docs)
print('Number of unique words in initital documents:', len(dictionary))

dictionary.filter_extremes(no_below=3, no_above=0.5)
print('Number of unique words after removing rare and common words:', len(dictionary))

Number of unique words in initital documents: 19839
Number of unique words after removing rare and common words: 5630


In [7]:
corpus = [dictionary.doc2bow(doc) for doc in docs]
print('Number of unique tokens: %d' % len(dictionary))
print('Number of documents: %d' % len(corpus))

Number of unique tokens: 5630
Number of documents: 2034


In [8]:
from gensim.models import LdaModel

num_topics = 25
chunksize = 500
passes = 20 
iterations = 40
eval_every = 1  

temp = dictionary[0]  
id2word = dictionary.id2token

%time model = LdaModel(corpus=corpus, id2word=id2word, chunksize=chunksize, \
                       alpha='auto', eta='auto', \
                       iterations=iterations, num_topics=num_topics, \
                       passes=passes, eval_every=eval_every)

Wall time: 49.9 s


In [9]:
#! pip install pyldavis
import pyLDAvis.gensim
pyLDAvis.enable_notebook()

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [10]:
pyLDAvis.gensim.prepare(model, corpus, dictionary)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.197960 -0.096478       1        1  16.911406
20    -0.214293 -0.079975       2        1  15.102233
5      0.003824  0.091749       3        1  13.429737
23    -0.097052 -0.149836       4        1   8.290062
21    -0.129561  0.131517       5        1   5.599470
6     -0.043651  0.291864       6        1   5.398775
13    -0.085356  0.168266       7        1   4.862653
16    -0.112803 -0.084982       8        1   4.117548
9     -0.155400 -0.140391       9        1   3.843047
0     -0.062290 -0.078248      10        1   3.747615
12    -0.009282  0.253273      11        1   2.553936
8      0.032546 -0.044402      12        1   2.473723
14    -0.098043  0.142986      13        1   2.321517
7     -0.129093 -0.092718      14        1   1.845839
24    -0.095035 -0.040512      15        1   1.760615
19     0.092384  0.047583      16        1   1.551694
10     0.141058  0.009780      17        1   1.447688
22    -0.061986 -0.037598      18        1   1.347280
2      0.153124  0.005638      19        1   0.873458
3      0.051125 -0.071846      20        1   0.649081
17     0.204456 -0.050531      21        1   0.589924
18     0.208698 -0.041360      22        1   0.463552
11     0.174004 -0.029614      23        1   0.356163
4      0.209976 -0.049853      24        1   0.279555
15     0.220610 -0.054311      25        1   0.183433, topic_info=          Term         Freq        Total Category  logprob  loglift
723        god   870.000000   870.000000  Default  30.0000  30.0000
284       imag   868.000000   868.000000  Default  29.0000  29.0000
60         n't  1820.000000  1820.000000  Default  28.0000  28.0000
214      space  1006.000000  1006.000000  Default  27.0000  27.0000
130      would  1286.000000  1286.000000  Default  26.0000  26.0000
...        ...          ...          ...      ...      ...      ...
3669  imprison     0.488571     6.135754  Topic25  -6.5043   3.7707
3743       sit     1.101563    19.601126  Topic25  -5.6913   3.4222
1069     becom     7.386222   152.823944  Topic25  -3.7885   3.2714
414       area     5.085696   150.160950  Topic25  -4.1616   2.9158
23        rule     2.127667   137.507599  Topic25  -5.0330   2.1324

[1452 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
2321     20  0.454784      a.m.
3674     12  0.925517   abolish
4377     17  0.961209     abort
399       1  0.988448   absolut
4471     22  0.844668  accommod
...     ...       ...       ...
789      19  0.987663      york
3606      5  0.163559      zero
3606      7  0.817795      zero
4044     22  0.934590   zillion
4345     18  0.923008      zone

[2865 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 21, 6, 24, 22, 7, 14, 17, 10, 1, 13, 9, 15, 8, 25, 20, 11, 23, 3, 4, 18, 19, 12, 5, 16])